<a href="https://colab.research.google.com/github/cbrumm/deep-learning-with-python-book/blob/main/Chapter_2_MNIST_digits_with_tensorflow_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf

class NaiveDense:
  def __init__(self, input_size, output_size, activiation):
    self.activiation = activiation

    w_shape = (input_size, output_size)
    w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
    self.W = tf.Variable(w_initial_value)

    b_initial_value = tf.zeros(output_size)
    self.b = tf.Variable(b_initial_value)

  def __call__(self, inputs):
    return self.activiation(tf.matmul(inputs, self.W) + self.b)

  @property
  def weights(self):
    return [self.W, self.b]

class NaiveSequential:
  def __init__(self, layers):
    self.layers = layers

  def __call__(self, inputs):
    x = inputs
    for layer in self.layers:
      x = layer(x)
    return x

  @property
  def weights(self):
    weights = []
    for layer in self.layers:
      weights += layer.weights
    return weights

In [7]:
model = NaiveSequential([
  NaiveDense(input_size=28*28, output_size=512, activiation=tf.nn.relu),
  NaiveDense(input_size=512, output_size=10, activiation=tf.nn.softmax)
])

assert len(model.weights) == 4

In [9]:
class BatchGenerator:
  def __init__(self, images, labels, batch_size=128):
    assert len(images) == len(labels)
    self.index = 0
    self.images = images
    self.labels = labels
    self.batch_size = batch_size
    self.num_batches = len(images) // batch_size

  def next(self):
    images = self.images[self.index : self.index + self.batch_size]
    labels = self.labels[self.index : self.index + self.batch_size]
    self.index += self.batch_size
    return images, labels

def one_training_step(model, images_batch, labels_batch):
  with tf.GradientTape() as tape:
    predictions = model(images_batch)
    per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(labels_batch, predictions)
    average_loss = tf.reduce_mean(per_sample_losses)

  gradients = tape.gradient(average_loss, model.weights)
  update_weights(gradients, model.weights)
  return average_loss

In [10]:
learning_rate = 1e-3

# Ordinarily, we would use a Keras optimizer instance for weight updates.
def update_weights(gradients, weights):
  for gradient, weight in zip(gradients, weights):
    weight.assign_sub(gradient * learning_rate)

In [11]:
def fit(model, images, labels, epochs, batch_size=128):
  for epoch_counter in range(epochs):
    print(f"Epoch {epoch_counter}")
    batch_generator = BatchGenerator(images, labels)
    for batch_counter in range(batch_generator.num_batches):
      images_batch, labels_batch = batch_generator.next()
      loss = one_training_step(model, images_batch, labels_batch)
      if batch_counter % 100 == 0:
        print(f"loss at batch {batch_counter}: {loss:.2f}")

In [12]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

11490434/11490434 [==============================] - 0s 0us/step
Epoch 0
loss at batch 0: 3.81
loss at batch 100: 2.22
loss at batch 200: 2.20
loss at batch 300: 2.10
loss at batch 400: 2.20
Epoch 1
loss at batch 0: 1.91
loss at batch 100: 1.87
loss at batch 200: 1.83
loss at batch 300: 1.72
loss at batch 400: 1.80
Epoch 2
loss at batch 0: 1.57
loss at batch 100: 1.57
loss at batch 200: 1.50
loss at batch 300: 1.44
loss at batch 400: 1.49
Epoch 3
loss at batch 0: 1.30
loss at batch 100: 1.34
loss at batch 200: 1.23
loss at batch 300: 1.22
loss at batch 400: 1.27
Epoch 4
loss at batch 0: 1.10
loss at batch 100: 1.16
loss at batch 200: 1.04
loss at batch 300: 1.06
loss at batch 400: 1.10
Epoch 5
loss at batch 0: 0.95
loss at batch 100: 1.02
loss at batch 200: 0.90
loss at batch 300: 0.95
loss at batch 400: 0.98
Epoch 6
loss at batch 0: 0.85
loss at batch 100: 0.92
loss at batch 200: 0.80
loss at batch 300: 0.86
loss at batch 400: 0.89
Epoch 7
loss at batch 0: 0.77
loss at batch 100: 0.83

In [14]:
import numpy as np

predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"accuracy: {matches.mean():.2f}")

accuracy: 0.83
